In [1]:
%matplotlib inline
from sympy import Symbol, Matrix,collect,expand,simplify
from sympy import init_printing
import numpy as np
from IPython.display import display
init_printing(use_latex=True)
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from models import LinearGaussianBN
from scipy.optimize import lsq_linear
from sklearn.linear_model import LinearRegression

In [2]:
def bounds_from_signs(signs):
    bounds_lower = [0 if s == 1 else -np.inf for s in signs]
    bounds_upper = [np.inf if s == 1 else 0 for s in signs]
    return (bounds_lower,bounds_upper)

def clip_to_bounds(intercept,coefficients,bounds):
    all_coef = np.concatenate(([intercept],coefficients))
    for indx,c in enumerate(all_coef):
        lower_bound = bounds[0][indx]
        upper_bound = bounds[1][indx]
        if c < lower_bound:
            all_coef[indx] = lower_bound
        elif c > upper_bound:
            all_coef[indx] = upper_bound
    return all_coef[0],all_coef[1:]

class ConstrainedLinearRegression(object):
    """ constrains value of coefficient during fit."""
    
    def __init__(self,signs):
        self.bounds = bounds_from_signs(signs)
    
    def fit(self,X,y):
        dataX = np.hstack((np.ones((X.shape[0],1)),X))
        result = lsq_linear(dataX,y,bounds = self.bounds)
        if not result.success:
            raise RuntimeError("Fit failed: "+result.message)
       
        self.intercept_, self.coef_ = clip_to_bounds(result.x[0],result.x[1:],self.bounds) # should already be within tolerance of bounds

class ClippedLinearRegression(object):
    """ sets each coefficient to closest valid point independently after fit"""
    
    def __init__(self,signs):
        self.model = LinearRegression()
        self.bounds = bounds_from_signs(signs)
    
    def fit(self,X,y):
        self.model.fit(X,y)
        self.intercept_,self.coef_ = clip_to_bounds(self.model.intercept_,self.model.coef_,self.bounds)

In [3]:
model = LinearGaussianBN()
model.add_var('Z')
model.add_var('A',['Z'])
model.add_var('B',['Z'])
model.add_var('C')
model.add_var('D')
model.add_var('Y',['Z','A','B','C','D'])

p_y = model.observe(['A','B','C','D'],['a','b','c','d']).marginal(['Y'])

In [4]:
w_AZ = 1
w_BZ = 1

w_YZ = -4
w_YA = 1
w_YB = 1
w_YC = .5
w_YD = -2.0

y_coeff = [0.1,w_YZ,w_YA,w_YB,w_YC,w_YD]
bounds = [1 if w > 0 else -1 for w in y_coeff]
bounds.pop(1) # remove bound on w_YZ as Z is latent

params = {"Z":([0],1),"A":([0,w_AZ],.1),"B":([0,w_BZ],.2),"C":([1],1),"D":([-1],.5),"Y":(y_coeff,1)}
p_y.set_params(params).parameterized_mean_cov()

In [5]:
n = 1000
data = model.marginal(['A','B','C','D','Y']).set_params(params).sample(n)
X = data[:,0:-1]
y = data[:,-1]
print(X.shape)

(1000, 4)


In [6]:
lfit = LinearRegression()
lfit.fit(X,y)
print(lfit.intercept_,lfit.coef_)
                 
cfit = ConstrainedLinearRegression(bounds)
cfit.fit(X,y)
print(cfit.intercept_,cfit.coef_)

clfit = ClippedLinearRegression(bounds)
clfit.fit(X,y)
print(clfit.intercept_,clfit.coef_)

0.301712029461 [-1.62611941 -0.13259358  0.43851399 -1.89039404]
0.288606464896 [  9.12105178e-20   7.44828320e-21   4.36151326e-01  -1.85270096e+00]
0.301712029461 [ 0.          0.          0.43851399 -1.89039404]


In [ ]:
# what is the query - are we looking at the value of the coefficients? Predictive accuracy given what intervention. When do we win?